# Selección de Características - Machine Learning

La selección de características es el proceso de elegir un subconjunto de las variables más importantes mientras se intenta retener la mayor cantidad de información posible

Veamos algunos métodos

## Métodos de filtrado

info dataset pima-indians-diabetes: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

In [ ]:
import pandas as pd

# Cargar data
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
df_classification = pd.read_csv(url, names=names)
print(df_classification.shape)

In [ ]:
df_classification.head(3)

In [ ]:
# Separación de caracteristicas y target
X_class = df_classification.drop(['class'], axis=1)
y_class = df_classification['class']

print(X_class.shape)
print(y_class.shape)

### Variance Threshold

Elimina todas las características cuya varianza no alcanza algún umbral. De forma predeterminada, elimina todas las características de varianza cero, es decir, las características que tienen el mismo valor en todas las muestras.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html

In [39]:
from sklearn.feature_selection import VarianceThreshold

#Función de filtro de caracteristicas
def variance_threshold(X,th):
    var_thres=VarianceThreshold(threshold=th)
    var_thres.fit(X)
    new_cols = var_thres.get_support()
    return new_cols

In [ ]:
# Obtener columnas seleccionadas
X_new_class = variance_threshold(X_class, 0.20)
# Nuevo dataframe
df_classification_new = X_class.iloc[:,X_new_class]
df_classification_new.head()

### SelectKBest

Selección de características de acuerdo con las k puntuaciones más altas. Utiliza una función que toma dos matrices X e y, y devuelve un par de matrices (puntuaciones, valores de p) o una única matriz con puntuaciones.

La función predeterminada solo funciona con tareas de clasificación (f_classif - ANOVA).

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest

In [41]:
# Para clasificación

from sklearn.feature_selection import SelectKBest, f_classif

# Función de filtro de caracteristicas - stadis. scores
def select_kbest_classification(X,y,score_f,k):
    sel_kb = SelectKBest(score_func=score_f, k=k)
    sel_kb.fit(X,y)
    new_cols = sel_kb.get_support()
    print("Scores:\n", sel_kb.scores_, "\nP-values:\n", sel_kb.pvalues_)
    return new_cols

In [ ]:
# Obtener columnas seleciconadas - (5 caracteristicas)
X_new_class = select_kbest_classification(X_class, y_class, f_classif, 5)
# Nuevo conjunto de datos
df_classification_new = X_class.iloc[:,X_new_class]
df_classification_new.head()

In [ ]:
# Para regresión
from sklearn.datasets import fetch_california_housing

# Lectura de datos
housing = fetch_california_housing(as_frame=True)
columns_drop = ["Longitude", "Latitude"]

X_reg = housing.data.drop(columns=columns_drop)
y_reg = housing.target

print("Feature data dimension: ", X_reg.shape)

In [ ]:
X_reg.head(3)
#y_reg.head()

In [50]:
from sklearn.feature_selection import SelectKBest, f_regression,r_regression

# Función de filtro de caracteristicas - stadis. scores
def select_kbest_regression(X,y,score_f,k):
    sel_kb = SelectKBest(score_func=score_f, k=k)
    sel_kb.fit(X,y)
    new_cols = sel_kb.get_support()
    #print("Scores:\n", sel_kb.scores_, "\nP-values:\n", sel_kb.pvalues_)
    print("Scores:\n", sel_kb.scores_)
    return new_cols

In [ ]:
# Obtener columnas seleciconadas - (3 caracteristicas)
X_new_reg = select_kbest_regression(X_reg, y_reg, f_regression, 3)
# Nuevo conjunto de datos
df_regression_new = X_reg.iloc[:,X_new_reg]
df_regression_new.head()

In [ ]:
# Obtener columnas seleciconadas - (3 caracteristicas)
X_new_reg = select_kbest_regression(X_reg, y_reg, r_regression, 3)
# Nuevo conjunto de datos
df_regression_new = X_reg.iloc[:,X_new_reg]
df_regression_new.head()

## Métodos Wrapper

### RFE

Dado un estimador externo que asigna pesos a las características (p. ej., los coeficientes de un modelo lineal), el objetivo de la eliminación recursiva de características (RFE) es seleccionar características considerando recursivamente conjuntos de características cada vez más pequeños.

Primero, el estimador se entrena en el conjunto inicial de características y la importancia de cada característica se obtiene a través de cualquier atributo específico o llamable. Luego, las características menos importantes se eliminan del conjunto actual de características. Ese procedimiento se repite recursivamente en el conjunto podado hasta que finalmente se alcanza el número deseado de características para seleccionar.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html

In [55]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, LogisticRegression

# Función recursiva de selección de características
def recursive_feature_selection(X,y,model,k):
  rfe = RFE(model, n_features_to_select=k, step=1)
  fit = rfe.fit(X, y)
  X_new = fit.support_
  print("Num Features: %s" % (fit.n_features_))
  print("Selected Features: %s" % (fit.support_))
  print("Feature Ranking: %s" % (fit.ranking_))

  return X_new

In [ ]:
# Para clasificación

# Establecer Estimador
model = LogisticRegression(max_iter=300)
# Obtener columnas seleciconadas - (3 caracteristicas)
X_new_class = recursive_feature_selection(X_class, y_class, model, 5)
# Nuevo conjunto de datos
df_classification_new = X_class.iloc[:,X_new_class]
df_classification_new.head()

In [ ]:
# Para regresión

# Establecer Estimador
model = LinearRegression()
# Obtener columnas seleciconadas - (3 caracteristicas)
X_new_reg = recursive_feature_selection(X_reg, y_reg, model, 3)
# Nuevo conjunto de datos
df_regression_new = X_reg.iloc[:,X_new_reg]
df_regression_new.head()


### SequentialFeatureSelector

Este selector secuencial de carcaterisicas agrega (selección hacia adelante) o elimina (selección hacia atrás) caracteristicas para formar un subconjunto de caracteristicas de manera codiciosa.

En cada etapa, este estimador elige la mejor característica para agregar o eliminar en función de la puntuación de validación cruzada de un estimador.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html


In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

# Selector secuencias utilizando regresión logistica - clasificación
sfs = SequentialFeatureSelector(LogisticRegression(),
                                n_features_to_select=5,
                                direction= "forward",
                                scoring='f1')

# Obtener variable seleccionadas
sfs = sfs.fit(X_class, y_class)
X_new_class = sfs.support_
df_classification_new = X_class.iloc[:,X_new_class]
df_classification_new.head()

In [ ]:
# Selector secuencias utilizando regresión lineal - Regresión
sfs = SequentialFeatureSelector(LinearRegression(),
                                n_features_to_select=3,
                                direction= "forward",
                                scoring='r2')

# Obtener variable seleccionadas
sfs = sfs.fit(X_reg, y_reg)
X_new_reg = sfs.support_
df_regression_new = X_reg.iloc[:,X_new_reg]
df_regression_new.head()

## Métodos integrados

### SelectFromModel

Meta-transformador para seleccionar características basadas en pesos de importancia.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import numpy as np

# Estandarizar los datos
standard_scaler = StandardScaler()
X_reg_std = standard_scaler.fit_transform(X_reg)
y_reg_std = standard_scaler.fit_transform(np.array(y_reg).reshape(-1, 1))

# Selector de variables con Lasso
sel_ = SelectFromModel(Lasso(alpha=0.03), max_features=4)
sel_.fit(X_reg_std, y_reg_std)
print(sel_.estimator_.coef_)
#Obtener variables seleccionadas
X_new_reg = sel_.get_support()

df_regression_new = X_reg.iloc[:,X_new_reg]
df_regression_new.head()

# Esquema de validación

### Cross Validation

Permite evaluar las métricas de desempeño de un modelo mediante validación cruzada y también registra los tiempos de entrenamiento/puntuación.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html

In [77]:
from sklearn.model_selection import  cross_validate

def cross_validation(model, _X, _y, _cv=5, scoring='f1'):
      _scoring = ['accuracy', 'precision', 'recall', 'f1']
      results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)

      return {"Training Accuracy scores": results['train_accuracy'],
              "Mean Training Accuracy": results['train_accuracy'].mean()*100,
              "Training Precision scores": results['train_precision'],
              "Mean Training Precision": results['train_precision'].mean(),
              "Training Recall scores": results['train_recall'],
              "Mean Training Recall": results['train_recall'].mean(),
              "Training F1 scores": results['train_f1'],
              "Mean Training F1 Score": results['train_f1'].mean(),
              "Validation Accuracy scores": results['test_accuracy'],
              "Mean Validation Accuracy": results['test_accuracy'].mean()*100,
              "Validation Precision scores": results['test_precision'],
              "Mean Validation Precision": results['test_precision'].mean(),
              "Validation Recall scores": results['test_recall'],
              "Mean Validation Recall": results['test_recall'].mean(),
              "Validation F1 scores": results['test_f1'],
              "Mean Validation F1 Score": results['test_f1'].mean()
              }

In [ ]:
# Modelo de regresión logística
log_model = LogisticRegression(class_weight="balanced", random_state=0, max_iter=300)
# Evaluación del modelo 2
log_model_2_result = cross_validation(log_model, X_class, y_class, 10)
print("Mean Training F1 Score: ", log_model_2_result['Mean Training F1 Score'],
      "\nMean Validation F1 Score: ", log_model_2_result['Mean Validation F1 Score'])

In [ ]:
# Otra alternativa utilizando LogisticRegressionCV
from sklearn.linear_model import LogisticRegressionCV

# Definición de modelo y ajuste a todos los datos
clf = LogisticRegressionCV(cv=10, random_state=0, class_weight="balanced", scoring='f1', max_iter=300).fit(X_class, y_class)

print("Score: ", clf.score(X_class, y_class))

### LeaveOneOut Cross-Validation

Proporciona índices de entrenamiento/prueba para dividir datos en conjuntos de entrenamiento/prueba. Cada muestra se usa una vez como conjunto de prueba (singleton) mientras que las muestras restantes forman el conjunto de entrenamiento.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html

In [ ]:
X_class = X_class.values
y_class = y_class.values

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score

# create loocv procedure
cv = LeaveOneOut()

# Listas de valores predichos y valores reales
y_true, y_pred = list(), list()

for train_ix, test_ix in cv.split(X_class):
  # split data
  X_train, X_test = X_class[train_ix, :], X_class[test_ix, :]
  y_train, y_test = y_class[train_ix], y_class[test_ix]
  # fit model
  model_log_3 = LogisticRegression(class_weight="balanced", random_state=0, max_iter=1000)
  model_log_3.fit(X_train, y_train)
  # evaluate model
  yhat = model_log_3.predict(X_test)
  # store
  y_true.append(y_test[0])
  y_pred.append(yhat[0])

# Metricas de evaluación
acc = accuracy_score(y_true, y_pred)
print('Accuracy: %.3f' % acc)
f1 = f1_score(y_true, y_pred)
print('F1 score: %.3f' % f1)